<a href="https://colab.research.google.com/github/mbozyk00/mt5-summarization-pl/blob/main/mt5_summarization_pl.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h4> $Polish$ $texts$ $summarization$ $with$ 🤗


<h4> $0.$ install libraries

In [ ]:
! pip install transformers datasets evaluate rouge_score sacrebleu py7zr -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 3.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.7/106.7 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.6/67.6 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 401.2/401.2 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 13.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 411.2/411.2 kB 11.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.9/138.9 kB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.7/49.7 kB 2.4 

In [ ]:
! pip install accelerate -U py7zr -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.6/302.6 kB 5.7 MB/s eta 0:00:00


In [ ]:
!nvidia-smi

Mon Aug 19 11:20:44 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA L4                      Off | 00000000:00:03.0 Off |                    0 |
| N/A   60C    P8              13W /  72W |      1MiB / 23034MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [ ]:
import pandas as pd
import torch
import evaluate
import numpy as np
import os
from transformers import pipeline
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, DataCollatorForSeq2Seq,
                        AutoConfig, Seq2SeqTrainer, Seq2SeqTrainingArguments
from datasets import load_dataset, load_metric
from torch.utils.data import DataLoader

from matplotlib import pyplot as plt

In [ ]:
torch.cuda.empty_cache()

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)

cuda


<h4> $1.$ Initialize tokenizer

In [ ]:
model_checkpoint = "google/mt5-small"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/82.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/553 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/4.31M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/99.0 [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
/usr/local/lib/python3.10/dist-packages/transformers/convert_slow_tokenizer.py:560: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


<h4> $2.$ Initialize model

In [ ]:
model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

pytorch_model.bin:   0%|          | 0.00/1.20G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

<h4> $3.$ Import dataset

In [ ]:
from datasets import load_dataset, DatasetDict

pol_corpus_tr = load_dataset("allegro/summarization-polish-summaries-corpus", split="train")
pol_corpus_val = load_dataset("allegro/summarization-polish-summaries-corpus", split="validation")
pol_corpus_te = load_dataset("allegro/summarization-polish-summaries-corpus", split="test")

Generating train split:   0%|          | 0/15598 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1734 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/4334 [00:00<?, ? examples/s]

In [ ]:
pol_corpus= DatasetDict({
    "train": pol_corpus_tr,
    "validation": pol_corpus_val,
    "test": pol_corpus_te,
})

In [ ]:
print(pol_corpus)

DatasetDict({
    train: Dataset({
        features: ['source', 'target'],
        num_rows: 15598
    })
    validation: Dataset({
        features: ['source', 'target'],
        num_rows: 1734
    })
    test: Dataset({
        features: ['source', 'target'],
        num_rows: 4334
    })
})


In [ ]:
print("Contents \n", pol_corpus["train"][1]["source"])
print("\nSummary \n", pol_corpus["train"][1]["target"])

Contents 
 WYMIAR SPRAWIEDLIWOŚCI 

Skarżący bacznie śledzą kolejność umieszczania spraw na wokandzie

Dokąd zmierzasz, kasacjo?

ANDRZEJ WYPIÓRKIEWICZ

Sąd Najwyższy ma wiele ustawowych obowiązków, nie można więc w nim widzieć tylko instancji do rozpatrywania kasacji.

Dokąd zmierzasz, kasacjo?Jest to pytanie o cel, ale bardziej o to, czy został osiągnięty w sprawach cywilnych i czy w ogóle jest to możliwe w warunkach funkcjonowania instytucji postępowania kasacyjnego wprowadzonych przez przepisy art. 392-39320 kodeksu postępowania cywilnego.

Cel jest jasny. Zapewnić obywatelom szeroki dostęp do niezależnego niezawisłego sądu (art. 45 konstytucji, art. 6 ust. 1 konwencji o ochronie praw człowieka i podstawowych wolności - Dz. U. z 1993 r. nr 61, poz. 284), w tym wypadku Sądu Najwyższego. To jedna z podstawowych zasad ustrojowych państwa prawnego i zasługuje na pełną społeczną aprobatę. Urzeczywistnianie tendencji do zwiększania podmiotowego i przedmiotowego zakresu spraw poddanych or

<h4> $4.$ Initial evaluation of pre-trained, not fine-tuned model on example entry

In [ ]:
contents = pol_corpus["test"][0]["source"]
target = pol_corpus["test"][0]["target"]
print(contents)

STUDIA

Rektorzy mówią o konieczności wprowadzenia częściowej odpłatności za studia

Początek roku akademickiego pod znakiem pieniądza

W piątek w wielu uczelniach odbyły się uroczyste inauguracje roku akademickiego, podczas których świeżo upieczeni studenci otrzymywali indeksy. Na zdjęciu studenci Uniwersytetu Warszawskiego: Michał Dec z prawa oraz Aleksandra Chrzanowska z kulturoznawstwa i Karolina Bogdan z polonistyki. 

BARTŁOMIEJ ZBOROWSKI

Trudno o akademik i stypendia

Studenci radzą sobie jak mogą

Szybciej przybywa studentów niż środków na pomoc materialną w budżecie państwa. Tę sytuację tylko trochę łagodzą wprowadzone w ubiegłym roku preferencyjne kredyty. Zdecydowanie brakuje miejsc w akademikach.

Co dziesiąty student Uniwersytetu Śląskiego, który składał podanie, nie otrzyma miejsca w akademiku, bo miejsc jest około 2,6 tysiąca na 38 tysięcy studiujących. Studenci, którzy otrzymają miejsce, płacić będą w zależności od dochodów w rodzinie od 50 do 80 proc. ceny akademika. 

In [ ]:
pipe = pipeline("summarization", model=model_checkpoint, max_new_tokens=256, tokenizer=tokenizer, device=-1)
pipe_out = pipe(contents)
print(pipe_out)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


[{'summary_text': '<extra_id_0>. - W tym roku można otrzymać tylko więcej.'}]


<h4> $5.$ Preprocess data - tokenization

In [ ]:
def tokenize_data(data):
    return {
        "input_ids": tokenizer(data["source"], truncation=True, padding="max_length", max_length=1024)["input_ids"],
        "attention_mask": tokenizer(data["source"], truncation=True, padding="max_length", max_length=1024)["attention_mask"],
        "labels": tokenizer(data["target"], truncation=True, padding="max_length", max_length=128)["input_ids"],
    }

In [ ]:
tokenized_corpus = pol_corpus.map(
    tokenize_data,
    remove_columns=["source", "target"],
    batched=True,
    batch_size=128
)
print(tokenized_corpus)

Map:   0%|          | 0/15598 [00:00<?, ? examples/s]

Map:   0%|          | 0/1734 [00:00<?, ? examples/s]

Map:   0%|          | 0/4334 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 15598
    })
    validation: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 1734
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 4334
    })
})


<h4> $6.$ Preprocess data - collator

In [ ]:
collator = DataCollatorForSeq2Seq(
    tokenizer,
    model=model
)

<h4> $7.$ ROUGE metrics

In [ ]:
rouge_metric = evaluate.load("rouge")

def tokenize_sentence(arg):
    encoded = tokenizer(arg, padding=True)
    return tokenizer.convert_ids_to_tokens(encoded.input_ids)

def metrics_calc(e_arg):
    preds, labels = e_arg

    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    preds = np.where(preds != -100, preds, tokenizer.pad_token_id)

    text_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
    text_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    return rouge_metric.compute(
        predictions=text_preds,
        references=text_labels,
        tokenizer=tokenize_sentence
    )

<h4> $8.$ Calculate ROUGE for vanilla model

In [ ]:
our_dataloader = DataLoader(
    tokenized_corpus["test"].with_format("torch"),
    collate_fn = collator,
    batch_size = 5
)

for batch in our_dataloader:
    with torch.no_grad():
        preds = model.generate(
            batch["input_ids"],
            max_new_tokens=50,
        )
    labels = batch["labels"]
    break
metrics_calc([preds, labels])

{'rouge1': 0.07668730430299411,
 'rouge2': 0.017957013746487428,
 'rougeL': 0.07657674598348059,
 'rougeLsum': 0.07668730430299411}

<h4> $9.$ Define training arguments

In [ ]:
training_args = Seq2SeqTrainingArguments(
    output_dir = "training_checkpoints",
    evaluation_strategy = "steps",
    per_device_train_batch_size = 2,
    per_device_eval_batch_size = 2,
    gradient_accumulation_steps = 16,
    weight_decay = 0.01,
    num_train_epochs = 10,
    learning_rate = 2e-3,
    lr_scheduler_type = "linear",
    warmup_steps = 90,
    log_level = "info",
    logging_strategy = "epoch",
    save_steps = 100,
    debug = "underflow_overflow",
    optim = "adafactor",
    predict_with_generate = True,
    generation_max_length = 128,
    push_to_hub = False
)

In [ ]:
trainer = Seq2SeqTrainer(
    model = model,
    args = training_args,
    data_collator = collator,
    compute_metrics = metrics_calc,
    train_dataset = tokenized_corpus["train"],
    eval_dataset = tokenized_corpus["validation"].select(range(20)),
    tokenizer = tokenizer,
)

<h4> $10.$ Train model

In [ ]:
trainer.train()

***** Running training *****
  Num examples = 15,598
  Num Epochs = 10
  Instantaneous batch size per device = 2
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 16
  Total optimization steps = 4,870
  Number of trainable parameters = 300,176,768


Step,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum
10,No log,12.717702,0.082514,0.024003,0.079577,0.079449
20,No log,3.693087,0.174073,0.032388,0.164654,0.165616
30,No log,2.575574,0.280948,0.083717,0.194893,0.195802
40,No log,2.544768,0.333016,0.133926,0.233361,0.233165
50,No log,2.863274,0.341313,0.146992,0.235207,0.235583
60,No log,3.239325,0.424962,0.242010,0.320011,0.319118
70,No log,2.985158,0.390998,0.217753,0.305480,0.304014
80,No log,2.550938,0.379132,0.177456,0.263377,0.264054
90,No log,2.673298,0.434858,0.262817,0.320463,0.320815
100,No log,2.402008,0.345611,0.142845,0.250449,0.248661


***** Running Evaluation *****
  Num examples = 20
  Batch size = 2
***** Running Evaluation *****
  Num examples = 20
  Batch size = 2
***** Running Evaluation *****
  Num examples = 20
  Batch size = 2
***** Running Evaluation *****
  Num examples = 20
  Batch size = 2
***** Running Evaluation *****
  Num examples = 20
  Batch size = 2
***** Running Evaluation *****
  Num examples = 20
  Batch size = 2
***** Running Evaluation *****
  Num examples = 20
  Batch size = 2
***** Running Evaluation *****
  Num examples = 20
  Batch size = 2
***** Running Evaluation *****
  Num examples = 20
  Batch size = 2
***** Running Evaluation *****
  Num examples = 20
  Batch size = 2
Saving model checkpoint to training_checkpoints\checkpoint-100
Configuration saved in training_checkpoints\checkpoint-100\config.json
Configuration saved in training_checkpoints\checkpoint-100\generation_config.json
Model weights saved in training_checkpoints\checkpoint-100\model.safetensors
tokenizer config file saved

TrainOutput(global_step=4870, training_loss=0.7543673568192938, metrics={'train_runtime': 630511.7114, 'train_samples_per_second': 0.247, 'train_steps_per_second': 0.008, 'total_flos': 1.648007036534784e+17, 'train_loss': 0.7543673568192938, 'epoch': 9.991024490319273})

<h4> $11.$ Save weights of fine-tuned model

In [ ]:
os.makedirs("./trained_for_summarization_pl", exist_ok=True)
if hasattr(trainer.model, "module"):
    trainer.model.module.save_pretrained("./trained_for_summarization_pl")
else:
    trainer.model.save_pretrained("./trained_for_summarization_pl")

model = (AutoModelForSeq2SeqLM
         .from_pretrained("./trained_for_summarization_pl"))

Configuration saved in ./trained_for_summarization_pl\config.json
Configuration saved in ./trained_for_summarization_pl\generation_config.json
Model weights saved in ./trained_for_summarization_pl\model.safetensors
loading configuration file ./trained_for_summarization_pl\config.json
Model config MT5Config {
  "_name_or_path": "./trained_for_summarization_pl",
  "architectures": [
    "MT5ForConditionalGeneration"
  ],
  "classifier_dropout": 0.0,
  "d_ff": 1024,
  "d_kv": 64,
  "d_model": 512,
  "decoder_start_token_id": 0,
  "dense_act_fn": "gelu_new",
  "dropout_rate": 0.1,
  "eos_token_id": 1,
  "feed_forward_proj": "gated-gelu",
  "initializer_factor": 1.0,
  "is_encoder_decoder": true,
  "is_gated_act": true,
  "layer_norm_epsilon": 1e-06,
  "model_type": "mt5",
  "num_decoder_layers": 8,
  "num_heads": 6,
  "num_layers": 8,
  "pad_token_id": 0,
  "relative_attention_max_distance": 128,
  "relative_attention_num_buckets": 32,
  "tie_word_embeddings": false,
  "tokenizer_class": "

<h4> $12.$ Evaluate model

In [ ]:
our_dataloader = DataLoader(
    tokenized_corpus["test"].with_format("torch"),
    collate_fn = collator,
    batch_size = 5
)

for batch in our_dataloader:
    with torch.no_grad():
        preds = model.generate(
            batch["input_ids"],
            max_new_tokens=50,
        )
    labels = batch["labels"]
    break
metrics_calc([preds, labels])

{'rouge1': 0.3550956993157251,
 'rouge2': 0.21314658817957305,
 'rougeL': 0.28319516970188374,
 'rougeLsum': 0.2811237366468339}

In [ ]:
sample_dataloader = DataLoader(
    tokenized_corpus["test"].with_format("torch"),
    collate_fn=collator,
    batch_size=5)
for batch in sample_dataloader:
    with torch.no_grad():
        preds = model.generate(
            batch["input_ids"],
            max_new_tokens=128,
        )
    labels = batch["labels"]
    break

labels = np.where(labels != -100, labels, tokenizer.pad_token_id)

text_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
text_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

print("***** Input's Text *****")
print(pol_corpus["test"]["source"][2])
print("***** Summary Text (True Value) *****")
print(text_labels[2])
print("***** Summary Text (Generated Text) *****")
print(text_preds[2])

***** Input's Text *****
LEKARZE

Odpowiedzialność cywilna

Od niedbalstwa do błędu

JAROSłAW CHAŁAS

Mimo upływu czasu wciąż jestem poruszony artykułem "Bezprecedensowy proces. Kardiochirurg twierdzi, że igły nie zagrażają życiu pacjentki" ("Rzeczpospolita" z 14 lutego 1997 r.)  Dlatego pragnę wrócić do  tematu. Nie od dziś dostrzegam znaczący wzrost zainteresowania tematyką nazwijmy to "lekarską". Jest, jak sądzę, kilka tego przyczyn. Należą do nich słusznie skądinąd nagłośnione przez media żądania środowiska lekarskiego dotyczące poprawy sytuacji finansowej w służbie zdrowia oraz odmawianie świadczenia usług leczniczych przez lekarzy państwowej służby zdrowia (zabiegi przerywania ciąży).

Wydaje mi się, że rozbudzi to w nas poczucie podmiotowości w stosunkach z lekarzami. Pacjent, którym przecież każdy z nas był i z pewnością będzie jeszcze nieraz, ma wiele praw, których respektowania może nie tylko oczekiwać, ale wręcz żądać. Czas zatem uświadomić sobie, że o swe prawa należy walcz